# Projekt Python
Tomasz Ancukiewicz 127219, Mikołaj Leśny 127218

18.11.2019

## Init libs

In [57]:
import pandas as pd
from scipy.io import arff
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
import numpy as np

## Load data

In [58]:
SEED = 42

# data, meta = arff.loadarff("data/breast-cancer.arff")
# df = pd.DataFrame(data)
df = pd.read_csv("data/breast-cancer.data", sep=',', 
                 names=['Class', 'age', 'menopause', 'tumor-size', 
                        'inv-nodes', 'node-caps', 'deg-malig', 
                        'breast', 'breast-quad', 'irradiat'])
# df = df.astype('str')

df[["age-begin", "age-end"]] = df.pop('age').astype('str').str.split("-", n=1, expand=True)
df[["tumor-size-begin", "tumor-size-end"]] = df.pop('tumor-size').astype('str').str.split("-", n=1, expand=True)
df[["inv-nodes-begin", "inv-nodes-end"]] = df.pop('inv-nodes').astype('str').str.split("-", n=1, expand=True)

df[["age-begin", "age-end", "tumor-size-begin", 
    "tumor-size-end", "inv-nodes-begin", "inv-nodes-end"]] = \
    df[["age-begin", "age-end", "tumor-size-begin", 
        "tumor-size-end", "inv-nodes-begin", "inv-nodes-end"]].astype(int)

y = df.pop('Class')
X = df
X = X.apply(lambda x: LabelEncoder().fit_transform(x))

## Prepared data

In [59]:
df.head()

,menopause,node-caps,deg-malig,breast,breast-quad,irradiat,age-begin,age-end,tumor-size-begin,tumor-size-end,inv-nodes-begin,inv-nodes-end
0,premeno,no,3,left,left_low,no,30,39,30,34,0,2
1,premeno,no,2,right,right_up,no,40,49,20,24,0,2
2,premeno,no,2,left,left_low,no,40,49,20,24,0,2
3,ge40,no,2,right,left_up,no,60,69,15,19,0,2
4,premeno,no,2,right,right_low,no,40,49,0,4,0,2


## Make test and train sets

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED)

## Classification

In [61]:
model = DecisionTreeClassifier(
    criterion='entropy',
    max_depth=7,
    min_samples_leaf=7,
    random_state=SEED,
    class_weight={"no-recurrence-events": 1.0, "recurrence-events": 5.0}
)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print(f'confusion matrix\n{tp, fp}\n{fn, tn}')
print(f'accuracy: {accuracy_score(y_test, y_pred)}')
recal_recorrence_events = classification_report(y_test, y_pred, output_dict=True)["recurrence-events"]["recall"]
print(f'recal[recurrence-events]: {recal_recorrence_events}')

confusion matrix
(18, 29)
(9, 30)
accuracy: 0.5581395348837209
recal[recurrence-events]: 0.6666666666666666
